# Señales continuas y discretas

> La información de una señal está contenida en un patrón de variaciones con una forma determinada.


## Introducción


**Ejemplo 1**: señal de voz humana (variaciones de presión acústica).


Distintos patrones de variación producen distintos sonidos:



In [69]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Span, HoverTool, Title
import numpy as np

output_notebook(verbose=False, hide_banner=True);

# ==== PARÁMETROS ====
sr = 16000  # frecuencia de muestreo
dur = 0.5   # segundos por letra
t_segment = np.linspace(0, dur, int(sr*dur), endpoint=False)

def envelope(t, attack=0.01, decay=0.25):
    env = np.ones_like(t)
    env *= np.minimum(1.0, t/attack)
    env *= np.exp(-t/decay)
    return env

# ==== SINTETIZAR LETRAS ====
# L
f0 = 120.0
harmonics = [1.0, 0.6, 0.3, 0.15]
sig_L = np.zeros_like(t_segment)
for i, a in enumerate(harmonics, start=1):
    sig_L += a * np.sin(2*np.pi*f0*i*t_segment)
sig_L *= envelope(t_segment, attack=0.02, decay=0.4)

# T
noise = np.random.normal(0, 1, size=t_segment.shape)
burst = np.zeros_like(t_segment)
burst_len = int(0.02*sr)
burst[:burst_len] = noise[:burst_len] * np.hanning(burst_len)
sig_T = burst * envelope(t_segment, attack=0.001, decay=0.1) + 0.02 * np.sin(2*np.pi*800*t_segment) * np.exp(-5*t_segment)

# I
f0_i = 160.0
formants = [2200, 3000]
sig_I = 0.6*np.sin(2*np.pi*f0_i*t_segment) * envelope(t_segment, attack=0.01, decay=0.5)
for fm in formants:
    sig_I += 0.3 * np.sin(2*np.pi*fm*t_segment) * envelope(t_segment, attack=0.01, decay=0.4)

# Normalizar
def norm(x): return x / (np.max(np.abs(x)) + 1e-9)
sig_L, sig_T, sig_I = map(norm, [sig_L, sig_T, sig_I])

# ==== CONCATENAR ====
gap = np.zeros(int(0.05*sr))
wave = np.concatenate([sig_L, gap, sig_T, gap, sig_I])
t_full = np.linspace(0, len(wave)/sr, len(wave), endpoint=False)

# ==== LÍMITES ====
start_L = 0
end_L = len(sig_L)
start_T = end_L + len(gap)
end_T = start_T + len(sig_T)
start_I = end_T + len(gap)
end_I = start_I + len(sig_I)

# ==== FUENTES DE DATOS ====
src_L = ColumnDataSource(data=dict(time=t_full[start_L:end_L], amp=sig_L, letter=["L"]*len(sig_L)))
src_T = ColumnDataSource(data=dict(time=t_full[start_T:end_T], amp=sig_T, letter=["T"]*len(sig_T)))
src_I = ColumnDataSource(data=dict(time=t_full[start_I:end_I], amp=sig_I, letter=["I"]*len(sig_I)))

# ==== GRAFICAR ====
TOOLS = "pan,wheel_zoom,box_zoom,reset,save,hover"
p = figure(title="Onda de audio: Pronunciación sintetizada de las letras L, T e I", 
            sizing_mode="stretch_width",
            max_width=800,
            height=400, 
            tools=TOOLS)

# Líneas por letra
p.line('time', 'amp', source=src_L, color="blue", line_width=2)
p.line('time', 'amp', source=src_T, color="green", line_width=2)
p.line('time', 'amp', source=src_I, color="red", line_width=2)

# Líneas divisorias
v1 = Span(location=end_L/sr, dimension='height', line_dash='dashed', line_width=1, line_color="gray")
v2 = Span(location=end_T/sr, dimension='height', line_dash='dashed', line_width=1, line_color="gray")
p.add_layout(v1)
p.add_layout(v2)

# Etiquetas ejes
p.xaxis.axis_label = "Tiempo (s)"
p.yaxis.axis_label = "Amplitud"
p.toolbar.logo = None

# ==== HOVER TOOL ====
hover = HoverTool(
    tooltips=[
        ("Letra", "@letter"),
        ("Tiempo (s)", "@time{0.000}"),
        ("Amplitud", "@amp{0.000}")
    ],
    mode="vline"  # muestra valores verticalmente alineados
)
p.add_tools(hover)

# ==== CAPTION (debajo) ====
# p.add_layout(Title(text="Distintos patrones de variación producen distintos sonidos.", 
#                    text_font_size="10pt", text_color="gray"), 'below')

# ==== MOSTRAR ====
show(p)


**Ejemplo 2**: fotografía (blanco y negro).

Señal bidimensional. Variaciones del nivel de gris.

In [ ]:
from PIL import Image
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, LinearColorMapper, ColorBar

output_notebook(verbose=False, hide_banner=True);

# ===== LOAD IMAGE =====
img = Image.open("figures\\cameraman ").convert("L")
img_array = np.array(img)
img_array = np.flipud(img_array)  # flip vertically

h, w = img_array.shape

# ===== NORMALIZE FOR COLOR MAPPING =====
color_mapper = LinearColorMapper(palette="Greys256", low=0, high=255);

# ===== CREATE FIGURE =====
p = figure(
    width=400, height=400,
    x_range=(0, w), y_range=(0, h),
    tools="hover,pan,box_zoom,reset,save"
);

# Remove ticks and grid
p.axis.visible = False
p.grid.visible = False

# Display image
p.image(image=[img_array], x=0, y=0, dw=w, dh=h, color_mapper=color_mapper);

# Add color bar
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12, location=(0,0));
# p.add_layout(color_bar, 'right')

# Hover tooltips (pixel coordinates)
hover = p.select_one(HoverTool);
hover.tooltips = [
    ("Intensity", "@image")  # @image references the value under the mouse
]
hover.mode = 'mouse';

show(p);


In [ ]:
from PIL import Image
import numpy as np
import plotly.graph_objects as go

# ===== LOAD IMAGE =====
img = Image.open("figures\\cameraman ").convert("L")
img_array = np.array(img)

# Flip vertically so top is at top
img_array = np.flipud(img_array)

h, w = img_array.shape
X, Y = np.meshgrid(np.arange(w), np.arange(h))
Z = img_array  # intensity as height

# ===== PLOTLY SURFACE =====
fig = go.Figure(data=[
    go.Surface(
        x=X,
        y=Y,
        z=Z,
        colorscale='gray',
        showscale=False,
        colorbar=dict(title='Intensity')
    )
])

fig.update_layout(
    # title='Variaciones del nivel de gris de la imagen',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Intensity',

        # <<--- camera / default view
        # camera=dict(eye=dict(x=.0, y=-.1, z=3))
    ),
    width=600,
    height=400,
    margin=dict(l=0, r=0, b=0, t=0)
)

fig.show()


---

**Representación matemática**: funciones de una o más variables independientes.

>Aquí sólo nos ocuparemos de señales con una variable independiente.

En general, consideramos que es el tiempo: $x(t)$ (aunque no tiene por qué).

Vamos a considerar **dos tipos básicos de señales**:
  
- **Continuas**: la variable independiente es continua $\Rightarrow$ se definen para una sucesión continua de valores de la variable independiente.

  **Notación**: $\boxed{x(t)}$, con $t\in\real$.
    
```{figure} figures/T1/2_1_fig4  
---
  width: 60%
---

```

    

```Ejemplos: señal de voz con el tiempo, presión atmosférica con altitud.

- **Discretas**: la variable independiente sólo toma un conjunto discreto de valores. También se las llama **secuencia discreta**.

  **Notación**: $\boxed{x[n]}$, con $n\in\Z$.

    
```{figure} figures/T1/2_1_fig5 
---
  width: 60%
---

```

    

Ejemplos: valor del IBEX-35 al final de cada sesión, muestreo de señales continuas (muestras equiespaciadas).

  

Vamos a ir viendo las señales continuas y discretas de forma paralela, para irlas relacionando. En el capítulo de muestreo (7), veremos cómo se puede pasar de unas a otras, idealmente sin error.

(clases_senales)=
## Clases de señales

Veremos a continuación distintos tipos de señales que tendrán importancia a lo largo de toda la asignatura.

  - **Real e imaginaria**: simetría respecto a la conjugación.
  
    - **Real pura**: simétrica respecto a la conjugación.

      \begin{equation}
	 x^*(t)=x(t),
      \end{equation}
      \begin{equation}
	 x^*[n]=x[n].
      \end{equation}

    - **Imaginaria pura**: antisimétrica respecto a la conjugación.

      \begin{equation}
	 x^*(t)=-x(t),
      \end{equation}
      \begin{equation}
	 x^*[n]=-x[n].
      \end{equation}

  

  - **Par e impar**: simetría respecto a la inversión en el tiempo. Se aplica a señales reales.
  
    - **Par**: simétrica respecto al eje de ordenadas.

      \begin{equation}
	 x(-t)=x(t),
      \end{equation}
      \begin{equation}
	 x[-n]=x[n].
      \end{equation}
    
    
      ```{figure} figures/T1/2_1_fig6 
      ---
      width: 60%
      ---

      ```

    
    - **Impar**: antisimétrica respecto al eje de ordenadas.

      \begin{equation}
    x(-t)=-x(t),
      \end{equation}
      \begin{equation}
    x[-n]=-x[n].
      \end{equation}
      
    
      ```{figure} figures/T1/2_1_fig7 
      ---
      width: 60%
      ---

      ```

          
      En el origen, como $x(0)=-x(0)$, se cumple:

        \begin{equation}
      x(0)=0,
        \end{equation}
        \begin{equation}
      x[0]=0,
        \end{equation}
    
  

- **Hermítica y antihermítica**: Equivalente para señales complejas.

  - **Hermítica**: simétrica respecto al eje de ordenadas y la conjugación.

      \begin{equation}
  x^*(-t)=x(t),
      \end{equation}
      \begin{equation}
  x^*[-n]=x[n].
      \end{equation}
    
  - **Antihermítica**: antisimétrica respecto al eje de ordenadas y la conjugación.

    \begin{equation}
x^*(-t)=-x(t),
    \end{equation}
    \begin{equation}
x^*[-n]=-x[n].
    \end{equation}
    
  

Toda señal se puede poner como suma de sus partes real e imaginaria, par e impar, hermítica y antihermítica:

  \begin{equation}
    x(t)=x_r(t)+x_i(t)=\Re\{x(t)\}+\Im\{x(t)\}.
  \end{equation}
  \begin{equation}
    x(t)=x_e(t)+x_o(t)=\mathcal{Ev}\{x(t)\}+\mathcal{Od}\{x(t)\},\quad x(t)\in\real.
  \end{equation}
  \begin{equation}
    x(t)=x_h(t)+x_a(t).
  \end{equation}
Las expresiones para el caso discreto son equivalentes.

Cálculo de la parte par e impar de una señal:

```{math}

\begin{cases}
   x(t)=x_e(t)+x_o(t),\
   x(-t)=x_e(-t)+x_o(-t)=x_e(t)-x_o(t).
  \end{cases}

```

```{math}
\Downarrow
```


  Parte par e impar de una señal:
  \begin{equation}
    \begin{cases}
      x_e(t)=\frac{1}{2}\left[x(t)+x(-t)\right],\
      x_o(t)=\frac{1}{2}\left[x(t)-x(-t)\right].
    \end{cases}
  \end{equation}

Se cumple:
```{math}
x_e(0)=x(0),\qquad x_e[0]=x[0].
```
```{math}
x_o(0)=0,\qquad x_o[0]=0.
```

Parte real e imaginaria:
  \begin{equation}
    \begin{cases}
      x_r(t)=\frac{1}{2}\left[x(t)+x^*(t)\right],\
      x_i(t)=\frac{1}{2}\left[x(t)-x^*(t)\right].
    \end{cases}
  \end{equation}

Parte hermítica y antihermítica:
  \begin{equation}
    \begin{cases}
      x_h(t)=\frac{1}{2}\left[x(t)+x^*(-t)\right],\
      x_a(t)=\frac{1}{2}\left[x(t)-x^*(-t)\right].
    \end{cases}
  \end{equation}


Para el caso discreto las expresiones son equivalentes.

**Ejemplo**: Cálculo de la parte par e impar de una señal:

$x[n]=\begin{cases}
  1, & n\ge 0,\
  0, & n<0.
\end{cases}$

 ```{figure} figures/T1/2_1_fig8 
 ---
 width: 60%
 ---

 ```

$x[-n]=\begin{cases}
  1, & n\le 0,\
  0, & n>0.
\end{cases}$

 ```{figure} figures/T1/2_1_fig9 
 ---
 width: 60%
 ---

 ```

$x_e[n]=\begin{cases}
  1/2, & n<0,\
  1, & n=0,\
  1/2, & n>0.
\end{cases}$

 ```{figure} figures/T1/2_1_fig10 
 ---
 width: 60%
 ---

 ```

$x_o[n]=\begin{cases}
  -1/2, & n<0,\
  0, & n=0,\
  1/2, & n>0.
\end{cases}$

 ```{figure} figures/T1/2_1_fig11 
 ---
 width: 60%
 ---
 ```

A continuación veremos otros tipos de señales: periódicas y de energía y de potencia.

(periodicas)=
## Señales periódicas


Dada su importancia las vemos aparte.

  - Una señal **continua**, $x(t)$, es **periódica** si:
  \begin{equation}\label{eq:periodica_continua}
    \exists\ T\in\real^+ /\quad \boxed{x(t)=x(t+T)},\quad \forall t\in\real.
  \end{equation}
  $T$: **periodo** de la señal.

  **Ejemplos**:
  
  ```{figure} figures/T1/2_1_fig12 
  ---
  width: 60%
  ---

  ```

  

  Si $x(t)$ es periódica con periodo $T$, también lo es con periodo $mT,\ m\in\N$.

  $T_0$: **periodo fundamental** de la señal. Valor más pequeño de $T$ para el que se satisface:
  \begin{equation}
    x(t)=x(t+T_0).
  \end{equation}

  Si $x(t)$ es constante, no está definido $T_0$, ya que es periódica para cualquier $T$.

  **Señal aperiódica**: si no es periódica.

  **Ejemplos:**
  ```{math}
  x(t)=\begin{cases}
    \cos(t), & t<0,\
    \sin(t), & t\ge 0.
  \end{cases}
  ```

  Se cumple que $\cos(t)=\cos(t+2\pi)$ para $t<-2\pi$ y $\sin(t)=\sin(t+2\pi)$, para $t\ge0$, pero no se cumple $x(t)=x(t+2\pi),\ \forall t$.

  
  ```{figure} figures/T1/2_1_fig13 
  ---
  width: 60%
  ---

  ```

  
 
  Estudiar el caso:
  ```{math}
  x(t)=\begin{cases}
    \cos(t), & t<0,\
    \cos(-t), & t\ge 0.
  \end{cases}
  ```

  - Una señal **discreta**, $x[n]$, es **periódica** si:
  \begin{equation}
    \exists\ N\in\N /\quad \boxed{x[n]=x[n+N]},\quad \forall n\in\Z.
  \end{equation}
  $N$: **periodo** de la señal.

  **Ejemplos**:
  
  ```{figure} figures/T1/2_1_fig14 
  ---
  width: 60%
  ---

  ```

  Si $x[n]$ es periódica con periodo $N$, también lo es con periodo $mN,\ m\in\N$.

  $N_0$: **periodo fundamental** de la secuencia discreta. Valor más pequeño de $N$ para el que se satisface:
  \begin{equation}
    x[n]=x[n+N_0].
  \end{equation}

  Si $x[n]$ es constante, $N_0=1$, que es el periodo mínimo que puede tener una señal discreta.

  **Señal aperiódica**: si no es periódica.
  

## Parámetros de interés

Vemos algunos parámetros de interés de las señales, tanto continuas como discretas.

  - **Valor medio**:
  
    - El **valor medio en un intervalo** viene dado por:
    
      - **Señales continuas**:\
  
	En el intervalo $t1\le t\le t_2$:
	\begin{equation}
	  \bar{x}\overset{\Delta}{=}\frac{1}{t_2-t_1}\int_{t_1}^{t_2} x(t)dt.
	\end{equation}
	Para un intervalo simétrico, $-T\le t\le T$:
	\begin{equation}
	  \bar{x}\overset{\Delta}{=}\frac{1}{2T}\int_{-T}^T x(t)dt.
	\end{equation}
      
      - **Señales discretas**:
  
      En el intervalo $n_1\le n\le n_2$:
	\begin{equation}
	  \bar{x}\overset{\Delta}{=}\frac{1}{n_2-n_1+1}\sum\limits_{n=n_1}^{n_2} x[n].
	\end{equation}
	Para un intervalo simétrico, $-N\le n\le N$:
	\begin{equation}
	  \bar{x}\overset{\Delta}{=}\frac{1}{2N+1}\sum\limits_{n=-N}^N x[n].
	\end{equation}
      
    
    - El **valor medio** viene dado por:
    
      - **Señales continuas**:
      \begin{equation}
	x_\text{AV}\overset{\Delta}{=}\lim_{T\to\infty}\frac{1}{T}\int_{-T/2}^{T/2} x(t)dt=\lim_{T\to\infty}\frac{1}{2T}\int_{-T}^{T} x(t)dt.
      \end{equation}
      - **Señales discretas**:
      \begin{equation}
	x_\text{AV}\overset{\Delta}{=}\lim_{N\to\infty}\frac{1}{2N+1}\sum\limits_{n=-N}^{N} x[n].
      \end{equation}
    
    - El **valor medio** para **señales periódicas** también se puede calcular de la siguiente forma:
    
      - **Señales continuas**:
      \begin{equation}
	x_\text{AV}\overset{\Delta}{=}\frac{1}{T}\int_{<T>} x(t)dt.
      \end{equation}
      - **Señales discretas**:
      \begin{equation}
	x_\text{AV}\overset{\Delta}{=}\frac{1}{N}\sum\limits_{n\in<N>} x[n].
      \end{equation}
    
  

  - **Valor de pico**:
  
    - **Señales continuas**:
    \begin{equation}
      x_p\overset{\Delta}{=}\max\{|x(t)|,\ t\in\real\}.
    \end{equation}
    - **Señales discretas**:
    \begin{equation}
      x_p\overset{\Delta}{=}\max\{|x[n]|,\ n\in\Z\}.
    \end{equation}
  

  - **Potencia instantánea**:\
  Por analogía con las señales que representan magnitudes físicas, se puede hablar de potencia y energía.

  Así por ejemplo, para una resistencia, la potencia instantánea:
  
```{figure} figures/T1/2_1_fig15 
---
  width: 60%
---

```

  
```{math}
p(t)=v(t)i(t)=\frac{1}{R}v^2(t)=Ri^2(t).
```
  Vemos que es proporcional a la señal al cuadrado. En otros ejemplos ocurre lo mismo.
 
  
  - **Señales continuas**:
  \begin{equation}
    P_i(t)\overset{\Delta}{=}|x(t)|^2.
  \end{equation}
  - **Señales discretas**:
  \begin{equation}
    P_i[n]\overset{\Delta}{=}|x[n]|^2.
  \end{equation}
  

  - **Energía**: suma (integral) de la potencia instantánea.
  
    - Energía total **en un intervalo** de tiempo:
    
      - **Señales continuas**:
  
        En el intervalo $t1\le t\le t_2$:
        \begin{equation}
          E\overset{\Delta}{=}\int_{t_1}^{t_2} |x(t)|^2dt.
        \end{equation}
        Para un intervalo simétrico, $-T\le t\le T$:
        \begin{equation}
          E_T\overset{\Delta}{=}\int_{-T}^T |x(t)|^2dt.
        \end{equation}
      
      - **Señales discretas**:
  
        En el intervalo $n_1\le n\le n_2$:
        \begin{equation}
          E\overset{\Delta}{=}\sum\limits_{n=n_1}^{n_2} |x[n]|^2.
        \end{equation}
        Para un intervalo simétrico, $-N\le n\le N$:
        \begin{equation}
          E_N\overset{\Delta}{=}\sum\limits_{n=-N}^N |x[n]|^2.
        \end{equation}
            
    
    - La **energía total** (en un intervalo infinito) viene dada por:
    
      - **Señales continuas**:
      \begin{equation}
	E_\infty\overset{\Delta}{=}\lim_{T\to\infty}\int_{-T}^{T} |x(t)|^2dt=\int_{-\infty}^{\infty} |x(t)|^2dt.
      \end{equation}
      - **Señales discretas**:
      \begin{equation}
	E_\infty\overset{\Delta}{=}\lim_{N\to\infty}\sum\limits_{n=-N}^{N} |x[n]|^2=\sum\limits_{n=-\infty}^{\infty} |x[n]|^2.
      \end{equation}
    

    Hay señales para las que esta integral (sumatorio) no converge, como por ejemplo:
    ```{math}
    x_1(t)=\frac{1}{\sqrt{t}},\qquad x_2(t)=C.
    ```
    ```{math}
    x_1[n]=\frac{1}{\sqrt{n}},\qquad x_2[n]=C.
    ```
    En estos casos, $E_\infty=\infty.$

    Por ello, nos va a interesar otra medida relacionada, que es la potencia media.
    
  - **Potencia media**: valor medio de la potencia instantánea.
  
    - En general, la potencia media viene dada por:
    
      - **Señales continuas**:
      \begin{equation}
	P_\infty\overset{\Delta}{=}\lim_{T\to\infty}\frac{1}{T}\int_{-T/2}^{T/2} |x(t)|^2dt=\lim_{T\to\infty}\frac{1}{2T}\int_{-T}^{T} |x(t)|^2dt.
      \end{equation}
      - **Señales discretas**:
      \begin{equation}
	P_\infty\overset{\Delta}{=}\lim_{N\to\infty}\frac{1}{2N+1}\sum\limits_{n=-N}^{N} |x[n]|^2.
      \end{equation}
    
    - En el caso particular de **señales periódicas**, también son aplicables las siguientes expresiones:
    
      - **Señales continuas**:
      \begin{equation}
	P_\infty\overset{\Delta}{=}\frac{1}{T}\int_{<T>} |x(t)|^2dt.
      \end{equation}
      - **Señales discretas**:
      \begin{equation}
	P_\infty\overset{\Delta}{=}\frac{1}{N}\sum\limits_{n\in<N>} |x[n]|^2.
      \end{equation}
    
  

**Ejemplo**: $x(t)=2.$

  ```{figure} figures/T1/2_1_fig16 
  ---
  width: 60%
  ---

  ```

```{math}
P_i(t)=|x(t)|^2=4.
```
```{math}
E_\infty=\int_{-\infty}^\infty |x(t)|^2 dt=\int_{-\infty}^\infty 4 dt=\infty.
```
```{math}
P_\infty=\lim_{T\to\infty}\frac{1}{T}\int_{-T/2}^{T/2} P_i(t) dt=\lim_{T\to\infty}\frac{4T}{T}=4.
```

## Señales de energía y de potencia

Según las definiciones de energía y de potencia de una señal, se puede hablar de tres clases de señales:

  - **Señales de energía**: son señales con **energía total finita**.
  ```{math}
  0<E_\infty<\infty.
  ```
  Tienen $P_\infty=0$:
  \begin{equation}
    P_\infty=\lim_{T\to\infty}\frac{1}{2T}\int_{-T}^{T} |x(t)|^2dt=\lim_{T\to\infty}\frac{E_T}{2T}=0.
  \end{equation}
  \begin{equation}
    P_\infty=\lim_{N\to\infty}\frac{1}{2N+1}\sum\limits_{n=-N}^{N} |x[n]|^2=\lim_{N\to\infty}\frac{E_N}{2N+1}=0.
  \end{equation}

  **Ejemplo**:

  ```{figure} figures/T1/2_1_fig17 
  ---
  width: 60%
  ---

  ```

  
  ```{math}
  E_\infty=1 \Rightarrow P_\infty=0.
  ```

  - **Señales de potencia**: son señales con **potencia media finita**.
  ```{math}
  0<P_\infty<\infty \quad \Rightarrow \quad E_\infty=\infty.
  ```
  Dado que tienen $P_\infty>0$, integrando (sumando) en un intervalo infinito, se obtiene $E_\infty=\infty$.

  **Ejemplos**:

  $x_1(t)=2.$\
  $x_2(t)$: cualquier señal periódica con valor de pico, $x_p$, finito.

  - **Señales con $E_\infty$ y $P_\infty$ infinitas**.

  **Ejemplo**:
  $x(t)=t.$
  ```{math}
  E_\infty=\lim_{T\to\infty}\int_{-T}^{T}|t|^2 dt=\lim_{T\to\infty}\int_{-T}^{T}t^2 dt=\lim_{T\to\infty}\frac{\left.t^3\right|_{-T}^{T}}{3}=
    \lim_{T\to\infty}\frac{2T^3}{3}=\infty.
  ```
  ```{math}
  P_\infty=\lim_{T\to\infty}\frac{1}{2T}\int_{-T}^{T}|t|^2 dt=\lim_{T\to\infty}\frac{E_T}{2T}=\lim_{T\to\infty}\frac{2T^3}{6T}=\infty.
  ```
